In [1]:
# Run MPNN on the inpainted regions
import os
os.chdir("/projects/crispy_shifty")

from crispy_shifty.utils.io import gen_array_tasks

design_list_file = os.path.join(os.getcwd(), "projects/DAB/01_inpaint_structure/inpainted_structures.list")

simulation_name = '02_mpnn_structure'
output_path = os.path.join(os.getcwd(), f"projects/DAB/{simulation_name}")

options = " ".join(
    [
        "out:level 100",
    ]
)
extra_kwargs = {
    "selection_name": "inpaint_seq_resis", 
    "df_scores": os.path.join(os.getcwd(), "projects/DAB/01_inpaint_structure/scores.csv"),
    "mpnn_design_area": "selection", 
    "num_sequences": "64", 
    "batch_size": "8",
    "omit_AAs": "CX",
}

gen_array_tasks(
    distribute_func="crispy_shifty.protocols.mpnn.mpnn_selection",
    design_list_file=design_list_file,
    output_path=output_path,
    queue="cpu",
    memory="5G",
    time="02:00:00",
    nstruct=1,
    nstruct_per_task=10,
    options=options,
    extra_kwargs=extra_kwargs,
    simulation_name=simulation_name,
)

Run the following command with your desired environment active:
sbatch -a 1-180 /home/broerman/crispy_shifty/projects/DAB/round_2/design/02_mpnn_structure/run.sh


In [1]:
import os
os.chdir("/projects/crispy_shifty")

from crispy_shifty.utils.io import collect_score_file

simulation_name = '02_mpnn_structure'
output_path = os.path.join(os.getcwd(), f"projects/DAB/{simulation_name}")

if not os.path.exists(os.path.join(output_path, "scores.json")):
    collect_score_file(output_path, "scores")

In [2]:
from crispy_shifty.utils.io import parse_scorefile_linear

scores_df = parse_scorefile_linear(os.path.join(output_path, "scores.json"))
# scores_df = scores_df.convert_dtypes()

  0%|          | 0/1800 [00:00<?, ?it/s]

In [3]:
from crispy_shifty.utils.io import df_to_fastas

scores_df = df_to_fastas(scores_df, prefix="mpnn_seq_")

  0%|          | 0/1800 [00:00<?, ?it/s]

In [4]:
from tqdm import tqdm

scores_df = scores_df.sort_values("total_length")

with open(os.path.join(output_path, "mpnn_inpaints.list"), "w") as f:
    for path in tqdm(scores_df.index):
        print(path, file=f)

100%|██████████| 1800/1800 [00:00<00:00, 1266311.17it/s]


In [5]:
with open(os.path.join(output_path, "mpnn_inpaints.pair"), "w") as f:
    for path in tqdm(scores_df.index):
        line = path + "____" + path.replace("decoys", "fastas").replace("pdb.bz2", "fa")
        print(line, file=f)

100%|██████████| 1800/1800 [00:00<00:00, 731848.31it/s]
